In [1]:
library(rethinking)
library(tidyverse)
data(Howell1)

Loading required package: rstan

Loading required package: StanHeaders

Loading required package: ggplot2

rstan (Version 2.21.2, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)

Loading required package: parallel

rethinking (Version 2.13)


Attaching package: ‘rethinking’


The following object is masked from ‘package:stats’:

    rstudent


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.5     ✔ dplyr   1.0.3
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks rstan::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ purrr::map()     masks rethinking::m

In [ ]:
d <- Howell1
d2 <- d[ d$age >= 18 , ]
xbar <- mean(d2$weight)

In [4]:
head(d2)

,height,weight,age,male
,<dbl>,<dbl>,<dbl>,<int>
1,151.765,47.82561,63,1
2,139.700,36.48581,63,0
3,136.525,31.86484,65,0
4,156.845,53.04191,41,1
5,145.415,41.27687,51,0
6,163.830,62.99259,35,1


In [5]:
m4.3 <- quap(
    alist(
        height ~ dnorm( mu , sigma ) ,
        mu <- a + b*( weight - xbar ) ,
        a ~ dnorm( 178 , 20 ) ,
        b ~ dlnorm( 0 , 1 ) ,
        sigma ~ dunif( 0 , 50 )
    ) , data=d2 )

In [6]:
dat <- data.frame( weight=c(45,40,65,31) )
h_sim <- sim( m4.3 , data=dat )
Eh <- apply(h_sim,2,mean)
h_ci <- apply(h_sim,2,PI,prob=0.89)

In [7]:
dat$Eh <- Eh
dat$L89 <- h_ci[1,]
dat$U89 <- h_ci[2,]
round(dat,1)

weight,Eh,L89,U89
<dbl>,<dbl>,<dbl>,<dbl>
45,154.5,146.4,162.3
40,150.0,142.0,158.1
65,172.6,164.5,181.1
31,141.8,133.3,150.2


In [9]:
PI

function (samples, prob = 0.89) 
{
    x <- sapply(prob, function(p) {
        a <- (1 - p)/2
        quantile(samples, probs = c(a, 1 - a))
    })
    n <- length(prob)
    result <- rep(0, n * 2)
    for (i in 1:n) {
        low_idx <- n + 1 - i
        up_idx <- n + i
        result[low_idx] <- x[1, i]
        result[up_idx] <- x[2, i]
        a <- (1 - prob[i])/2
        names(result)[low_idx] <- concat(round(a * 100, 0), "%")
        names(result)[up_idx] <- concat(round((1 - a) * 100, 
            0), "%")
    }
    return(result)
}
<bytecode: 0x7f8896a4b708>
<environment: namespace:rethinking>